In [1]:
import pandas as pd
from sqlalchemy.exc import SQLAlchemyError # Para capturar errores específicos de SQLAlchemy si es necesario
from src.conexion_bd import ConexionBD  # Ajusta la ruta si es necesario

print("--- Demostración del Sistema de Análisis de Ventas ---")

--- Demostración del Sistema de Análisis de Ventas ---


In [2]:
print("\n--- 1. Probando Conexión a la Base de Datos (Singleton) ---")
try:
    conector_bd1 = ConexionBD()
    motor1 = conector_bd1.obtener_motor()

    conector_bd2 = ConexionBD()
    motor2 = conector_bd2.obtener_motor()

    if motor1 and motor2:
        print(f"Instancia 1 del conector: {id(conector_bd1)}")
        print(f"Instancia 2 del conector: {id(conector_bd2)}")
        print(f"¿Son la misma instancia? {'Sí' if conector_bd1 is conector_bd2 else 'No'}")
        print(f"Motor de la instancia 1: {motor1}")
        print(f"Motor de la instancia 2: {motor2}")
        print(f"¿Son el mismo motor? {'Sí' if motor1 is motor2 else 'No'}")
        
        # Probar una conexión simple
        with motor1.connect() as connection:
            print("Conexión directa al motor exitosa.")
            
    else:
        print("Error: No se pudo obtener el motor de la base de datos. Revisa las credenciales y la configuración.")

except ValueError as ve:
    print(f"Error de configuración: {ve}")
except Exception as e:
    print(f"Error inesperado al conectar: {e}")


--- 1. Probando Conexión a la Base de Datos (Singleton) ---
INFO     | sistema_ventas | Sistema de logging configurado correctamente
INFO     | sistema_ventas | Nivel de logging: INFO
INFO     | sistema_ventas | Directorio de logs: d:\gnavarro\Escritorio\sistema-de-analisis-de-ventas\logs
INFO     | sistema_ventas.GestorCache | Cache de consultas inicializado - Habilitado: True
INFO     | sistema_ventas.GestorCache | Configuración: duración=15min, tamaño_max=100
INFO     | sistema_ventas.ConexionBD | INICIANDO: inicialización de conexión a base de datos
INFO     | sistema_ventas.ConexionBD | Conexión a la base de datos establecida exitosamente.
INFO     | sistema_ventas.ConexionBD | Pool configurado: size=5, overflow=10, query_timeout_config=30s
INFO     | sistema_ventas.Estadisticas | MÉTRICAS BD - Conexiones activas: 1, Consultas ejecutadas: 0
INFO     | sistema_ventas.ConexionBD | Cache inicializado - Habilitado: True
INFO     | sistema_ventas.ConexionBD | COMPLETADO: inicializació

In [3]:
print("\n--- 2. Resultados de Consultas SQL ---")

# (conector_bd1 fue instanciado en una celda anterior del notebook)
if 'conector_bd1' in locals() and conector_bd1.obtener_motor(): # Verifica si el motor está disponible
    print("\nConsulta: Primeros 5 clientes")

    df_clientes = conector_bd1.ejecutar_consulta("SELECT * FROM Customers LIMIT 5;") 
    if df_clientes is not None:
        display(df_clientes)
    else:
        print("No se pudieron obtener los clientes.")

    print("\nConsulta: Productos con precio (Price) mayor a 50")
    # Usamos "Products" para la tabla y "ProductName", "Price" para las columnas.
    df_productos_caros = conector_bd1.ejecutar_consulta(
        "SELECT ProductName, Price FROM Products WHERE Price > 50 ORDER BY Price DESC LIMIT 5;" 
    )
    if df_productos_caros is not None:
        display(df_productos_caros)
    else:
        print("No se pudieron obtener los productos caros.")
else:
    print("No se pueden ejecutar consultas, la conexión a la BD no está establecida o `conector_bd1` no está definido.")


--- 2. Resultados de Consultas SQL ---

Consulta: Primeros 5 clientes
INFO     | sistema_ventas.GestorCacheConsultas | INICIANDO: búsqueda en cache de consulta
INFO     | sistema_ventas.GestorCacheConsultas | COMPLETADO: búsqueda en cache de consulta (duración: 0.008s)
INFO     | sistema_ventas.ConexionBD | HIT DE CACHE para consulta: SELECT * FROM Customers LIMIT 5;... (0.0220s)
INFO     | sistema_ventas.ConexionBD | Consulta exitosa: 5 filas en 0.026s


,CustomerID,FirstName,MiddleInitial,LastName,Address,CityID
0,1,Stefanie,Y,Frye,97 Oak Avenue,79
1,2,Sandy,T,Kirby,52 White First Freeway,96
2,3,Lee,T,Zhang,921 White Fabien Avenue,55
3,4,Regina,S,Avery,75 Old Avenue,40
4,5,Daniel,S,Mccann,283 South Green Hague Avenue,2



Consulta: Productos con precio (Price) mayor a 50
INFO     | sistema_ventas.GestorCacheConsultas | INICIANDO: búsqueda en cache de consulta
INFO     | sistema_ventas.GestorCacheConsultas | COMPLETADO: búsqueda en cache de consulta (duración: 0.001s)
INFO     | sistema_ventas.ConexionBD | HIT DE CACHE para consulta: SELECT ProductName, Price FROM Products WHERE Price > 50 ORD... (0.0084s)
INFO     | sistema_ventas.ConexionBD | Consulta exitosa: 5 filas en 0.010s


,ProductName,Price
0,Shrimp - 31/40,99.8755
1,Beef - Inside Round,99.3193
2,Puree - Passion Fruit,98.8263
3,Bread - Calabrese Baguette,98.5978
4,Zucchini - Yellow,98.4644


In [4]:
#Análisis Avanzado - Ranking de Productos por Ventas Dentro de Cada Categoría (Cálculo de ventas DESDE Products.Price CON DESCUENTO)
print("\n--- 3.1. Análisis Avanzado: Top Productos por Categoría (Cálculo desde Products.Price con Descuento) ---")

if 'conector_bd1' in locals() and conector_bd1.obtener_motor():
    # Asumimos que Sales.Discount es un decimal (ej. 0.10 para 10%)
    # Usamos IFNULL(s.Discount, 0) para tratar los NULL como sin descuento (0%)
    # Usamos p.Price de la tabla Products como precio base.
    consulta_avanzada_sql = """
    WITH VentasProducto AS (
        -- Paso 1: Calcular el monto total de ventas netas para cada producto
        SELECT
            p.ProductID,
            p.ProductName,
            p.CategoryID,
            SUM(s.Quantity * p.Price * (1 - IFNULL(s.Discount, 0))) AS monto_total_ventas_producto
        FROM Sales s
        JOIN Products p ON s.ProductID = p.ProductID
        GROUP BY
            p.ProductID,
            p.ProductName,
            p.CategoryID
    ),
    VentasProductoCategoriaConNombre AS (
        -- Paso 2: Unir con la tabla de categorías para obtener el nombre de la categoría
        SELECT
            vp.ProductID,
            vp.ProductName,
            vp.CategoryID,
            cat.CategoryName,
            vp.monto_total_ventas_producto
        FROM VentasProducto vp
        JOIN Categories cat ON vp.CategoryID = cat.CategoryID
    ),
    RankingVentas AS (
        -- Paso 3: Rankear los productos dentro de cada categoría según su monto total de ventas netas
        SELECT
            vpc.ProductID,
            vpc.ProductName,
            vpc.CategoryName,
            vpc.monto_total_ventas_producto,
            RANK() OVER (PARTITION BY vpc.CategoryName ORDER BY vpc.monto_total_ventas_producto DESC) AS ranking_en_categoria
        FROM VentasProductoCategoriaConNombre vpc
    )
    -- Paso 4: Seleccionar el resultado final, mostrando por ejemplo el Top 3 de productos por categoría
    SELECT
        CategoryName AS "Categoría",
        ProductName AS "Producto",
        monto_total_ventas_producto AS "Monto Total de Ventas Netas",
        ranking_en_categoria AS "Ranking en Categoría"
    FROM RankingVentas
    WHERE ranking_en_categoria <= 3  -- Puedes ajustar este número
    ORDER BY
        "Categoría",
        ranking_en_categoria;
    """
    
    print("\nConsulta Avanzada: Top 3 Productos por Ventas Netas en Cada Categoría (Usando Products.Price y Sales.Discount)")
    # print("SQL a ejecutar:\n", consulta_avanzada_sql) # Descomenta esto para ver el SQL exacto que se envía
    df_ranking_productos = conector_bd1.ejecutar_consulta(consulta_avanzada_sql)
    
    if df_ranking_productos is not None and not df_ranking_productos.empty:
        print("¡Consulta ejecutada exitosamente!")
        display(df_ranking_productos)
    elif df_ranking_productos is not None and df_ranking_productos.empty:
        print("La consulta se ejecutó correctamente pero no arrojó resultados.")
    else:
        print("No se pudo obtener el ranking de productos. Revisa el mensaje de error anterior de la base de datos.")
else:
    print("No se puede ejecutar la consulta avanzada, la conexión a la BD no está establecida o `conector_bd1` no está definido.")


--- 3.1. Análisis Avanzado: Top Productos por Categoría (Cálculo desde Products.Price con Descuento) ---

Consulta Avanzada: Top 3 Productos por Ventas Netas en Cada Categoría (Usando Products.Price y Sales.Discount)
INFO     | sistema_ventas.ConexionBD | Consulta exitosa: 33 filas en 0.010s
¡Consulta ejecutada exitosamente!


,Categoría,Producto,Monto Total de Ventas Netas,Ranking en Categoría
0,Beverages,Placemat - Scallop; White,191.57300,1
1,Produce,Orange - Canned; Mandarin,157.68100,1
2,Snails,Cookie Dough - Double,222.19829,1
3,Cereals,Shrimp - 31/40,199.75100,1
4,Grain,Bread - Multigrain,193.04000,1
5,Shell fish,Lettuce - Spring Mix,90.01690,1
6,Confections,Juice - Lime,183.56660,1
7,Meat,Cod - Black Whole Fillet,290.45320,1
8,Seafood,Cheese - Wine,148.81520,1
9,Dairy,Scampi Tail,380.38280,1


In [5]:
# --- 4. Demostración del Patrón Factory (FabricaModelos) ---
print("\\n--- 4. Demostración del Patrón Factory (FabricaModelos) ---")

from src.utils.fabrica_modelo import FabricaModelos #
from src.modelos.cliente import Cliente #

# Crear una instancia de la fábrica
fabrica = FabricaModelos()

# Datos de ejemplo para crear un cliente
datos_cliente_ejemplo = {
    'CustomerID': 999,
    'FirstName': 'Gustavo',
    'MiddleInitial': 'A',
    'LastName': 'Navarro',
    'CityID': 1, # Asume que CityID 1 existe
    'Address': 'Calle Demo 123'
}

print(f"\\nCreando un objeto Cliente usando FabricaModelos con datos: {datos_cliente_ejemplo}")

try:
    cliente_creado = fabrica.create_from_dict('cliente', datos_cliente_ejemplo) #
    print("\\nCliente creado exitosamente:")
    print(f"  ID: {cliente_creado.id_cliente}") #
    print(f"  Nombre Completo: {cliente_creado.nombre_completo()}") #
    print(f"  Dirección: {cliente_creado.direccion}") #
    print(f"  Tipo de objeto: {type(cliente_creado)}")
except ValueError as ve:
    print(f"Error al crear cliente con la fábrica: {ve}")
except Exception as e:
    print(f"Un error inesperado ocurrió: {e}")

# Ejemplo con un DataFrame 
import pandas as pd
datos_df_ejemplo = pd.DataFrame([
    {
        'CustomerID': 1000, 'FirstName': 'Ana', 'LastName': 'López', 'CityID': 1,
        'MiddleInitial': 'B', 'Address': 'Calle Mayor 123'
    }
])
print(f"\\nCreando objetos Cliente desde un DataFrame usando FabricaModelos:")
try:
    clientes_desde_df = fabrica.create_multiple_from_dataframe('cliente', datos_df_ejemplo) #
    if clientes_desde_df:
        print("Clientes creados desde DataFrame:")
        for cliente_df in clientes_desde_df:
            print(f"  - {cliente_df.nombre_completo()}, ID: {cliente_df.id_cliente}")
    else:
        print("No se crearon clientes desde el DataFrame.")
except ValueError as ve:
    print(f"Error al crear clientes desde DataFrame: {ve}")
except Exception as e:
    print(f"Un error inesperado ocurrió: {e}")

\n--- 4. Demostración del Patrón Factory (FabricaModelos) ---
INFO     | sistema_ventas.FabricaModelos | FabricaModelos inicializada correctamente
\nCreando un objeto Cliente usando FabricaModelos con datos: {'CustomerID': 999, 'FirstName': 'Gustavo', 'MiddleInitial': 'A', 'LastName': 'Navarro', 'CityID': 1, 'Address': 'Calle Demo 123'}
INFO     | sistema_ventas.FabricaModelos | INICIANDO: creación de modelo desde diccionario
INFO     | sistema_ventas.FabricaModelos | Modelo cliente creado exitosamente (ID: 999)
INFO     | sistema_ventas.FabricaModelos | COMPLETADO: creación de modelo desde diccionario (duración: 0.007s)
\nCliente creado exitosamente:
  ID: 999
  Nombre Completo: Gustavo A. Navarro
  Dirección: Calle Demo 123
  Tipo de objeto: <class 'src.modelos.cliente.Cliente'>
\nCreando objetos Cliente desde un DataFrame usando FabricaModelos:
INFO     | sistema_ventas.FabricaModelos | INICIANDO: creación múltiple de modelos desde DataFrame
INFO     | sistema_ventas.FabricaModelos 

In [6]:
# --- 5. Demostración del Patrón Builder (ConstructorConsultaSQL) ---
print("\\n--- 5. Demostración del Patrón Builder (ConstructorConsultaSQL) ---")

from src.utils.constructor_consulta import ConstructorConsultaSQL #

# Crear una instancia del constructor
constructor_sql = ConstructorConsultaSQL()

# Construir una consulta paso a paso
print("\\nConstruyendo una consulta SQL para 'Empleados de la ciudad con ID 7':")
consulta_construida = (constructor_sql
                       .seleccionar("EmployeeID", "FirstName", "LastName", "HireDate") #
                       .desde_tabla("employees") #
                       .donde("CityID = 7") #
                       .ordenar_por("HireDate", "DESC") #
                       .limite(5) #
                       .construir()) #

print("\\nConsulta SQL construida:")
print(consulta_construida)

# Opcionalmente, ejecutar la consulta construida (asumiendo que conector_bd1 ya está definido y conectado)
print("\\nEjecutando la consulta construida (si la conexión está activa):")
if 'conector_bd1' in locals() and conector_bd1.obtener_motor():
    try:
        df_resultado_builder = conector_bd1.ejecutar_consulta(consulta_construida)
        if df_resultado_builder is not None:
            display(df_resultado_builder)
        else:
            print("La consulta construida no devolvió resultados o hubo un error.")
    except Exception as e:
        print(f"Error al ejecutar la consulta construida: {e}")
else:
    print("No se puede ejecutar la consulta, la conexión a la BD no está establecida o `conector_bd1` no está definido.")
    
# LÍNEA DE DEPURACIÓN A AGREGAR:
print(f"DEBUG: Tipo de constructor_sql ANTES de llamar a .reiniciar(): {type(constructor_sql)}")

# Demostrar reinicio y construcción de otra consulta
print("\\nDemostrando reutilización del constructor después de reiniciar:")
consulta_productos = (constructor_sql.reiniciar() #
                      .seleccionar("ProductName", "Price")
                      .desde_tabla("products")
                      .donde("Price < 10")
                      .ordenar_por("Price", "ASC")
                      .limite(3)
                      .construir())
print("\\nNueva consulta SQL construida para productos:")
print(consulta_productos)

\n--- 5. Demostración del Patrón Builder (ConstructorConsultaSQL) ---
\nConstruyendo una consulta SQL para 'Empleados de la ciudad con ID 7':
\nConsulta SQL construida:
SELECT EmployeeID, FirstName, LastName, HireDate
FROM employees
WHERE CityID = 7
ORDER BY HireDate DESC
LIMIT 5;
\nEjecutando la consulta construida (si la conexión está activa):
INFO     | sistema_ventas.GestorCacheConsultas | INICIANDO: búsqueda en cache de consulta
INFO     | sistema_ventas.GestorCacheConsultas | COMPLETADO: búsqueda en cache de consulta (duración: 0.001s)
INFO     | sistema_ventas.ConexionBD | HIT DE CACHE para consulta: SELECT EmployeeID, FirstName, LastName, HireDate
FROM employ... (0.0065s)
INFO     | sistema_ventas.ConexionBD | Consulta exitosa: 1 filas en 0.010s


,EmployeeID,FirstName,LastName,HireDate
0,23,Janet,Flowers,2010-05-12


DEBUG: Tipo de constructor_sql ANTES de llamar a .reiniciar(): <class 'src.utils.constructor_consulta.ConstructorConsultaSQL'>
\nDemostrando reutilización del constructor después de reiniciar:
\nNueva consulta SQL construida para productos:
SELECT ProductName, Price
FROM products
WHERE Price < 10
ORDER BY Price ASC
LIMIT 3;


In [7]:
# --- 6. Ejecución de Pruebas Unitarias (pytest) ---
print("\\n--- 6. Ejecución de Pruebas Unitarias (pytest) ---")
print("Ejecutando pytest para todas las pruebas en el directorio 'tests/'...")
print("La salida de las pruebas se mostrará a continuación:")

# Usar ! para ejecutar comandos de shell.
# El modificador -v es para salida verbosa, --tb=short para un traceback más corto.
!pytest tests/ -v --tb=short

\n--- 6. Ejecución de Pruebas Unitarias (pytest) ---
Ejecutando pytest para todas las pruebas en el directorio 'tests/'...
La salida de las pruebas se mostrará a continuación:
============================= test session starts =============================
platform win32 -- Python 3.10.11, pytest-7.4.3, pluggy-1.6.0 -- D:\gnavarro\Escritorio\sistema-de-analisis-de-ventas\venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: d:\gnavarro\Escritorio\sistema-de-analisis-de-ventas
configfile: pytest.ini
plugins: anyio-4.9.0, cov-4.1.0, mock-3.12.0
collecting ... collected 22 items

tests/test_conexion_bd.py::test_singleton_conexion_bd PASSED             [  4%]
tests/test_conexion_bd.py::test_ejecutar_consulta_sin_conexion PASSED    [  9%]
tests/test_empleado.py::test_empleado_describir_antiguedad_varios_casos PASSED [ 13%]
tests/test_integridad_datos.py::test_conteo_paises PASSED                [ 18%]
tests/test_integridad_datos.py::test_conteo_ventas PASSED                [ 22%]
tests/te